<a href="https://colab.research.google.com/github/urvashiramdasani/Document-Summarization/blob/main/notebooks/20_newsgroups_using_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook presents the code for 20 Newsgroups dataset document summarization.

## Setting up GitHub

In [1]:
!git clone https://github.com/urvashiramdasani/Document-Summarization.git

Cloning into 'Document-Summarization'...
remote: Enumerating objects: 18944, done.
remote: Counting objects: 100% (18944/18944), done.
remote: Compressing objects: 100% (18909/18909), done.
remote: Total 18944 (delta 56), reused 18893 (delta 33), pack-reused 0
Receiving objects: 100% (18944/18944), 19.52 MiB | 18.35 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (18855/18855), done.


## Data Preprocessing

In [2]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    filedata = filedata[15:]
    sentences = []

    for sentence in filedata:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").strip(" \n"))
    
    return sentences

In [22]:
# Reading a sample article

sentences = read_article("/content/Document-Summarization/data/20news-bydate-train/sci.space/59905")
sentences = list(filter(None, sentences))
print(sentences)

['This section was lightly adapted from an original posting by Larry Klaes', '(klaes@verga.enet.dec.com), mostly minor formatting changes. Matthew', 'Wiener (weemba@libra.wistar.upenn.edu) contributed the section on', 'Voyager, and the section on Sakigake was obtained from ISAS material', 'posted by Yoshiro Yamada (yamada@yscvax.ysc.go.jp).', 'US PLANETARY MISSIONS', 'MARINER (VENUS, MARS, & MERCURY FLYBYS AND ORBITERS)', 'MARINER 1, the first U.S. attempt to send a spacecraft to Venus, failed', 'minutes after launch in 1962. The guidance instructions from the ground', 'stopped reaching the rocket due to a problem with its antenna, so the', 'onboard computer took control. However, there turned out to be a bug in', 'the guidance software, and the rocket promptly went off course, so the', 'Range Safety Officer destroyed it. Although the bug is sometimes claimed', 'to have been an incorrect FORTRAN DO statement, it was actually a', 'transcription error in which the bar (indicating smoothi

In [6]:
len(sentences)

414

## POS Tagging

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
txt = ""
for sentence in sentences:
  txt += "".join(sentence)
  txt += " "
print(txt)

This section was lightly adapted from an original posting by Larry Klaes (klaes@verga.enet.dec.com), mostly minor formatting changes. Matthew Wiener (weemba@libra.wistar.upenn.edu) contributed the section on Voyager, and the section on Sakigake was obtained from ISAS material posted by Yoshiro Yamada (yamada@yscvax.ysc.go.jp). US PLANETARY MISSIONS MARINER (VENUS, MARS, & MERCURY FLYBYS AND ORBITERS) MARINER 1, the first U.S. attempt to send a spacecraft to Venus, failed minutes after launch in 1962. The guidance instructions from the ground stopped reaching the rocket due to a problem with its antenna, so the onboard computer took control. However, there turned out to be a bug in the guidance software, and the rocket promptly went off course, so the Range Safety Officer destroyed it. Although the bug is sometimes claimed to have been an incorrect FORTRAN DO statement, it was actually a transcription error in which the bar (indicating smoothing) was omitted from the expression "R-dot-b

In [ ]:
tokenized = sent_tokenize(txt)
tagged = []
for i in tokenized:
    wordsList = nltk.word_tokenize(i)
    wordsList = [w for w in wordsList if not w in stop_words] 
    tagged.append(nltk.pos_tag(wordsList))

In [ ]:
print(tagged)

[[('This', 'DT'), ('section', 'NN'), ('lightly', 'RB'), ('adapted', 'VBD'), ('original', 'JJ'), ('posting', 'NN'), ('Larry', 'NNP'), ('Klaes', 'NNP'), ('(', '('), ('klaes', 'VB'), ('@', 'NNP'), ('verga.enet.dec.com', 'NN'), (')', ')'), (',', ','), ('mostly', 'RB'), ('minor', 'JJ'), ('formatting', 'NN'), ('changes', 'NNS'), ('.', '.')], [('Matthew', 'NNP'), ('Wiener', 'NNP'), ('(', '('), ('weemba', 'JJ'), ('@', 'NNP'), ('libra.wistar.upenn.edu', 'NN'), (')', ')'), ('contributed', 'VBD'), ('section', 'NN'), ('Voyager', 'NNP'), (',', ','), ('section', 'NN'), ('Sakigake', 'NNP'), ('obtained', 'VBD'), ('ISAS', 'NNP'), ('material', 'NN'), ('posted', 'VBD'), ('Yoshiro', 'NNP'), ('Yamada', 'NNP'), ('(', '('), ('yamada', 'PRP'), ('@', 'NNP'), ('yscvax.ysc.go.jp', 'NN'), (')', ')'), ('.', '.')], [('US', 'NNP'), ('PLANETARY', 'NNP'), ('MISSIONS', 'NNP'), ('MARINER', 'NNP'), ('(', '('), ('VENUS', 'NNP'), (',', ','), ('MARS', 'NNP'), (',', ','), ('&', 'CC'), ('MERCURY', 'NNP'), ('FLYBYS', 'NNP'), (

In [ ]:
annotations = set()

for i in range(len(tagged)):
  for j in range(len(tagged[i])):
    annotations.add(tagged[i][j][1])

print(annotations)

{'POS', 'CD', 'VBZ', 'DT', 'FW', '.', 'PRP', 'JJR', '``', "''", 'VBP', 'MD', 'VBN', '$', ',', 'WRB', 'JJS', 'PRP$', 'NN', '#', 'NNP', 'NNPS', 'RB', 'VBG', 'JJ', 'IN', '(', 'RBR', 'NNS', 'RP', 'VBD', 'CC', ')', 'VB', ':', 'WP'}


In [ ]:
tagged[0][0][1]

'NNP'

In [ ]:
corpus = ""
for i in range(len(tagged)):
  for j in range(len(tagged[i])):
    if tagged[i][j][1] != 'JJ' and tagged[i][j][1] != 'DT':
      corpus += tagged[i][j][0] + " "

print(corpus)

section lightly adapted posting Larry Klaes ( klaes @ verga.enet.dec.com ) , mostly formatting changes . Matthew Wiener ( @ libra.wistar.upenn.edu ) contributed section Voyager , section Sakigake obtained ISAS material posted Yoshiro Yamada ( yamada @ yscvax.ysc.go.jp ) . US PLANETARY MISSIONS MARINER ( VENUS , MARS , & MERCURY FLYBYS AND ORBITERS ) MARINER 1 , U.S. attempt send spacecraft Venus , failed minutes 1962 . guidance instructions ground stopped reaching rocket problem antenna , onboard computer took control . However , turned guidance software , rocket promptly went course , Range Safety Officer destroyed . Although bug sometimes claimed FORTRAN DO statement , actually transcription error bar ( indicating smoothing ) omitted expression `` sub n '' ( nth smoothed value radius ) . error led software treat variations velocity , compensation . MARINER 2 became first probe flyby Venus December 1962 , returned information confirmed Venus ( 800 degrees Fahrenheit , revised 900 degr

## Hidden Markov Models



In [5]:
CONST = 2
n = 4
threshold = 

In [49]:
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils, to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [56]:
# Preprocess data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab = tokenizer.word_index
seqs = tokenizer.texts_to_sequences(sentences)

In [57]:
def prepare_sentence(seq, maxlen):
    # Pads seq and slides windows
    x = []
    y = []
    for i, w in enumerate(seq):
        x_padded = pad_sequences([seq[:i]],
                                 maxlen=maxlen - 1,
                                 padding='pre')[0]  # Pads before each sequence
        x.append(x_padded)
        y.append(w)
    return x, y

# Pad sequences and slide windows
maxlen = max([len(seq) for seq in seqs])
x = []
y = []
for seq in seqs:
    x_windows, y_windows = prepare_sentence(seq, maxlen)
    x += x_windows
    y += y_windows
x = np.array(x)
y = np.array(y) - 1  # The word <PAD> does not constitute a class
y = np.eye(len(vocab))[y]  # One hot encoding

In [59]:
# Define model
model = Sequential()
model.add(Embedding(input_dim=len(vocab) + 1,  # vocabulary size. Adding an
                                               # extra element for <PAD> word
                    output_dim=5,  # size of embeddings
                    input_length=maxlen - 1))  # length of the padded sequences
model.add(LSTM(10))
model.add(Dense(len(vocab), activation='softmax'))
model.compile('rmsprop', 'categorical_crossentropy', metrics = ['accuracy'])

# Train network
model.fit(x, y, epochs=100)

Epoch 1/100
130/130 [==============================] - 3s 6ms/step - loss: 6.9858 - accuracy: 0.0341
Epoch 2/100
130/130 [==============================] - 1s 6ms/step - loss: 6.3703 - accuracy: 0.0587
Epoch 3/100
130/130 [==============================] - 1s 7ms/step - loss: 6.1174 - accuracy: 0.0645
Epoch 4/100
130/130 [==============================] - 1s 6ms/step - loss: 6.1413 - accuracy: 0.0640
Epoch 5/100
130/130 [==============================] - 1s 6ms/step - loss: 6.1329 - accuracy: 0.0568
Epoch 6/100
130/130 [==============================] - 1s 6ms/step - loss: 6.1083 - accuracy: 0.0591
Epoch 7/100
130/130 [==============================] - 1s 6ms/step - loss: 6.0378 - accuracy: 0.0660
Epoch 8/100
130/130 [==============================] - 1s 6ms/step - loss: 6.0503 - accuracy: 0.0622
Epoch 9/100
130/130 [==============================] - 1s 6ms/step - loss: 6.0317 - accuracy: 0.0569
Epoch 10/100
130/130 [==============================] - 1s 6ms/step - loss: 5.9767 - accura

In [62]:
# Compute probability of occurence of a sentence
def probability_of_sentence(sentence):
  tok = tokenizer.texts_to_sequences([sentence])[0]
  x_test, y_test = prepare_sentence(tok, maxlen)
  x_test = np.array(x_test)
  y_test = np.array(y_test) - 1  # The word <PAD> does not constitute a class
  p_pred = model.predict(x_test)  # array of conditional probabilities
  vocab_inv = {v: k for k, v in vocab.items()}

  # Compute product
  # Efficient version: np.exp(np.sum(np.log(np.diag(p_pred[:, y_test]))))
  log_p_sentence = 0
  for i, prob in enumerate(p_pred):
      word = vocab_inv[y_test[i]+1]  # Index 0 from vocab is reserved to <PAD>
      history = ' '.join([vocab_inv[w] for w in x_test[i, :] if w != 0])
      prob_word = prob[y_test[i]]
      log_p_sentence += np.log(prob_word)
      # print('P(w={}|h={})={}'.format(word, history, prob_word))
  return np.exp(log_p_sentence)

In [63]:
sent = "VENERA 1 - First acknowledged attempt at Venus mission. Transmissions 	       lost enroute in 1961.  	       February of 1966. No images were returned.   	       became the first craft to impact on another planet in 1966. VENERA 4 - First probe to successfully return data while descending"
probability_of_sentence(sent)

4.522007576165454e-94

In [83]:
import pandas as pd

df = pd.DataFrame(columns = ['Sentence', 'Probability'])

In [84]:
# txt = txt.split(" .")
word_list = ['', '', '', '', '', '', '', '']

for i in range(len(sentences)):
  words = sentences[i].split(" .")
  num_words = len(words)
  for j in range(num_words):
    word_list[3] = words[j]
    if j + n <= num_words:
      word_list[5] = words[j + n]
    if j - n >= 0:
      word_list[2] = words[j - n]
    if j + 1 < num_words:
      word_list[4] = words[j + 1]
    if i - CONST >= 0:
      word_list[0] = sentences[i - CONST].split(" .")[j]
    if i - CONST + 1>= 0:
      word_list[1] = sentences[i - CONST + 1].split(" .")[j]
    if i + CONST < len(sentences):
      word_list[7] = sentences[i + CONST].split(" .")[j]
    if i + CONST - 1 < len(sentences):
      word_list[6] = sentences[i + CONST - 1].split(" .")[j]
    final_str = " ".join(word_list)
    df = df.append({'Sentence':final_str, 'Probability':probability_of_sentence(final_str)}, ignore_index = True)

In [85]:
df

,Sentence,Probability
0,This section was lightly adapted from an or...,6.322890e-92
1,"(klaes@verga.enet.dec.com), mostly minor fo...",1.954026e-66
2,Wiener (weemba@libra.wistar.upenn.edu) cont...,5.532167e-56
3,This section was lightly adapted from an orig...,1.013105e-112
4,This section was lightly adapted from an origi...,2.017713e-148
...,...,...
409,"periodical, THE PLANETARY REPORT, details the ...",3.110970e-125
410,"missions. Write to The Planetary Society, 65 N...",1.389112e-134
411,"Pasadena, California 91106 USA. Good luck with...",5.487490e-128
412,Good luck with your studies in this area of sp...,3.291456e-135


In [86]:
df.describe()

,Probability
count,4.140000e+02
mean,1.336272e-58
std,2.718912e-57
min,5.415781e-176
25%,5.300537e-147
50%,7.114659e-131
75%,6.191777e-110
max,5.532167e-56


In [87]:
# Filter out all the sentences whose probability > 1.0e-140
df = df[df.Probability > 1.0e-120]

In [88]:
df

,Sentence,Probability
0,This section was lightly adapted from an or...,6.322890e-92
1,"(klaes@verga.enet.dec.com), mostly minor fo...",1.954026e-66
2,Wiener (weemba@libra.wistar.upenn.edu) cont...,5.532167e-56
3,This section was lightly adapted from an orig...,1.013105e-112
38,MARINER 5 was sent to Venus in 1967. It reconf...,2.941806e-104
...,...,...
391,some of the books listed above. Some periodica...,4.189552e-120
393,"GEOGRAPHIC, which has written articles on the ...",1.373068e-118
397,"VOYAGER missions to Jupiter, Saturn, Uranus, a...",2.213013e-115
404,"affected by numerous political, economic, and ...",7.952822e-106


## BERT

In [10]:
!pip install bert-extractive-summarizer==0.4.2

     |████████████████████████████████| 2.1MB 5.5MB/s 
     |████████████████████████████████| 901kB 17.4MB/s 
     |████████████████████████████████| 3.3MB 29.0MB/s 
  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.4.2-cp37-none-any.whl size=13711 sha256=3e427b2ab5a1effe3059db1b884a85056eb4c771306df23f87f8a87930a6a2cb
  Stored in directory: /root/.cache/pip/wheels/13/bc/30/654eb9e657177a56cba927c5a20b6cd01fb229b1ed2bf9b371
Successfully built bert-extractive-summarizer


In [12]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 4.2MB/s 


In [14]:
import torch
from summarizer import Summarizer

In [15]:
model = Summarizer('distilbert-base-uncased')

In [22]:
resp = model(txt)
print(resp)

This section was lightly adapted from an original posting by Larry Klaes (klaes@verga.enet.dec.com), mostly minor formatting changes. The guidance instructions from the ground stopped reaching the rocket due to a problem with its antenna, so the onboard computer took control. Although the bug is sometimes claimed to have been an incorrect FORTRAN DO statement, it was actually a transcription error in which the bar (indicating smoothing) was omitted from the expression "R-dot-bar sub n" (nth smoothed value of derivative of radius). The probe found a cratered world with an atmosphere much thinner than previously thought. However, they failed to take away the concept of Mars as a "dead" planet, first made from the basic measurements of MARINER 4. MARINER 9, the sister probe to MARINER 8, became the first craft to orbit Mars in 1971. The probe did return the first close-up images of the Venusian atmosphere in ultraviolet, revealing previously unseen details in the cloud cover, plus the fac

In [23]:
print(len(resp), len(txt))

4832 24938
